In [ ]:
# !pip install langchain
# !pip install openai
# !pip install PyPDF2
# !pip install faiss-cpu
# !pip install tiktoken

In [ ]:
'''
create a page on gitmind for this 
 1.read the pdfs
2. Break the pdfs into components or blocks
3. Create embedings 
4. Use a database for the embedings - Vectorstore
'''

from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

import pandas as pd

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "add your key"


In [ ]:
# provide the path of  pdf file/files.
pdfreader = PdfReader(r'..\fake_cv.pdf')

In [ ]:
# notes: we need to read in all the pages 
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:

# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [ ]:
document_search = FAISS.from_texts(texts, embeddings)

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(model= 'gpt-3.5-turbo-instruct'), chain_type="stuff")

In [ ]:
query = " how many companies worked in. and for how many years "
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

In [ ]:
query = "what are the python libraries you know "
docs = document_search.similarity_search(query)
print(chain.run(input_documents=docs, question=query))

In [ ]:
def generate_text_from_pdf(pdf_file ):
    #step 1
    pdfreader = PdfReader(pdf_file)
    #step 2
    # read text from pdf
    raw_text = ''
    for i, page in enumerate(pdfreader.pages):
        content = page.extract_text()
        if content:
            raw_text += content
    #step 3
    # We need to split the text using Character Text Split such that it sshould not increse token size
    text_splitter = CharacterTextSplitter(
        separator = "\n",
        chunk_size = 800,
        chunk_overlap  = 200,
        length_function = len,
    )
    texts = text_splitter.split_text(raw_text)

    #step 4
    embeddings = OpenAIEmbeddings(model= 'gpt-3.5-turbo-instruct')

    #step 5
    document_search = FAISS.from_texts(texts, embeddings)

    return document_search


In [ ]:
candidates= ['candidate_1', 'candidate_2']
Hr_question_1 = 'which companies have you worked in and for how long'
Hr_question_2 = 'what are the python libraries you know.enumerate 3' 
Hr_question_3 = 'email and contact number'


In [ ]:
# build database to store 
df = pd.DataFrame(columns= [f'{Hr_question_1}',f'{Hr_question_2}' ,f'{Hr_question_3}'  ] , index= candidates)


In [ ]:
# gernate FAISS embeding documnent search for each document
document_search_1 = generate_text_from_pdf(pdf_file=r'C:\Users\gghotr01\OneDrive - Teranet\Code\Langchain\data\CV_Jasprit_Labana_Ghotra.pdf')
document_search_2 = generate_text_from_pdf(pdf_file=r'C:\Users\gghotr01\OneDrive - Teranet\Code\Langchain\data\fake_cv.pdf')
# run query on each for each candidate
# update the dataframe as we process it 
document_search_list = [document_search_1 , document_search_2]

In [ ]:
for candidate, document_search  in zip(candidates,document_search_list) :
    print(candidate , document_search)
    for query in [Hr_question_1, Hr_question_2, Hr_question_3]:
        print(query)
        docs = document_search.similarity_search(query)
        df.loc[candidate , query] = chain.run(input_documents=docs, question=query)  
    

In [ ]:
print(df)